# The Battle of Neighbourhoods
## Business Problem
My client would like to open a Japanese restaurant in North York, Toronto. He would like the restaurant to be located in a neighbourhood of North York (e.g. Parkwoods, Victoria Village, etc.) that has the fewest number of restaurants as a common venue.

I will perform the following steps to help my client pick a neighbourhood:
* I will obtain data for North York's neighbourhoods from this URL: https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M
* I will explore and analyze North York's neighbourhoods by using Foursquare data.
* I will determine which neighbourhood has the fewest number of restaurants.

In [57]:
#import libraries
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

#!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

#!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

print('Libraries imported.')

ModuleNotFoundError: No module named 'folium'

### Obtain a dataframe of North York's neighbourhoods

In [79]:
import requests

url = 'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
results = requests.get(url)

# Ignore cells with a borough that is Not assigned.
df_raw = pd.read_html(results.content, header=0)[0]
df_new = df_raw[df_raw.Borough != 'Not assigned']
df_new

,Postal Code,Borough,Neighbourhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"
5,M6A,North York,"Lawrence Manor, Lawrence Heights"
6,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"
8,M9A,Etobicoke,"Islington Avenue, Humber Valley Village"
9,M1B,Scarborough,"Malvern, Rouge"
11,M3B,North York,Don Mills
12,M4B,East York,"Parkview Hill, Woodbine Gardens"
13,M5B,Downtown Toronto,"Garden District, Ryerson"


In [80]:
# Create a new dataframe with only North York
df_NorthYork = df_new.loc[df_new['Borough'] == 'North York']
df_NorthYork.rename(columns = {'Neighbourhood':'Neighborhood'}, inplace = True)
df_NorthYork

/opt/conda/envs/Python-3.7-main/lib/python3.7/site-packages/pandas/core/frame.py:4133: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


,Postal Code,Borough,Neighborhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
5,M6A,North York,"Lawrence Manor, Lawrence Heights"
11,M3B,North York,Don Mills
14,M6B,North York,Glencairn
20,M3C,North York,Don Mills
46,M2H,North York,Hillcrest Village
47,M3H,North York,"Bathurst Manor, Wilson Heights, Downsview North"
55,M2J,North York,"Fairview, Henry Farm, Oriole"
56,M3J,North York,"Northwood Park, York University"


In [81]:
df_NorthYork.shape

(24, 3)

In [82]:
#Obtain Longitude and Latitude for different Postal Codes

url = 'http://cocl.us/Geospatial_data'
df_geo=pd.read_csv(url)
df_geo.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [83]:
NorthYork_data = pd.merge(df_geo, df_NorthYork, on='Postal Code')
NorthYork_data.head()

,Postal Code,Latitude,Longitude,Borough,Neighborhood
0,M2H,43.803762,-79.363452,North York,Hillcrest Village
1,M2J,43.778517,-79.346556,North York,"Fairview, Henry Farm, Oriole"
2,M2K,43.786947,-79.385975,North York,Bayview Village
3,M2L,43.757490,-79.374714,North York,"York Mills, Silver Hills"
4,M2M,43.789053,-79.408493,North York,"Willowdale, Newtonbrook"


### Explore neighbourhoods in North York with Foursquare data

In [84]:
CLIENT_ID = 'VNYVHNWG0DVEUYVTNZKLWEAMDSSJLWEHI5YGFDRCI3H5XSIR' # your Foursquare ID
CLIENT_SECRET = 'MCMF2FD3MQASSB4HLRQFHQGBYWIWOEXIE4ZUXYNWVDLYWTNU' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version
LIMIT = 100 # A default Foursquare API limit value

In [64]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [85]:
NorthYork_venues = getNearbyVenues(names=NorthYork_data['Neighborhood'],
                                   latitudes=NorthYork_data['Latitude'],
                                   longitudes=NorthYork_data['Longitude']
                                  )

Hillcrest Village
Fairview, Henry Farm, Oriole
Bayview Village
York Mills, Silver Hills
Willowdale, Newtonbrook
Willowdale, Willowdale East
York Mills West
Willowdale, Willowdale West
Parkwoods
Don Mills
Don Mills
Bathurst Manor, Wilson Heights, Downsview North
Northwood Park, York University
Downsview
Downsview
Downsview
Downsview
Victoria Village
Bedford Park, Lawrence Manor East
Lawrence Manor, Lawrence Heights
Glencairn
North Park, Maple Leaf Park, Upwood Park
Humber Summit
Humberlea, Emery


In [86]:
print(NorthYork_venues.shape)
NorthYork_venues.head()

(246, 7)


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Hillcrest Village,43.803762,-79.363452,New York Fries,43.803664,-79.363905,Fast Food Restaurant
1,Hillcrest Village,43.803762,-79.363452,Eagle's Nest Golf Club,43.805455,-79.364186,Golf Course
2,Hillcrest Village,43.803762,-79.363452,AY Jackson Pool,43.804515,-79.366138,Pool
3,Hillcrest Village,43.803762,-79.363452,Villa Madina,43.801685,-79.363938,Mediterranean Restaurant
4,Hillcrest Village,43.803762,-79.363452,Duncan Creek Park,43.805539,-79.360695,Dog Run


In [87]:
NorthYork_venues.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
"Bathurst Manor, Wilson Heights, Downsview North",22,22,22,22,22,22
Bayview Village,4,4,4,4,4,4
"Bedford Park, Lawrence Manor East",26,26,26,26,26,26
Don Mills,25,25,25,25,25,25
Downsview,13,13,13,13,13,13
"Fairview, Henry Farm, Oriole",66,66,66,66,66,66
Glencairn,5,5,5,5,5,5
Hillcrest Village,5,5,5,5,5,5
Humber Summit,4,4,4,4,4,4


**Anaylze each neighbourhood**

In [88]:
# one hot encoding
NorthYork_onehot = pd.get_dummies(NorthYork_venues[['Venue Category']], prefix="", prefix_sep="")
NorthYork_onehot.insert(loc=0, column='Neighborhood', value=NorthYork_venues['Neighborhood'] )
NorthYork_onehot.shape


(246, 104)

In [89]:
NorthYork_grouped = NorthYork_onehot.groupby('Neighborhood').mean().reset_index()
NorthYork_grouped.head()

,Neighborhood,Accessories Store,Airport,American Restaurant,Art Gallery,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Bakery,Bank,Bar,Baseball Field,Basketball Court,Beer Store,Bike Shop,Boutique,Breakfast Spot,Bridal Shop,Bubble Tea Shop,Burger Joint,Burrito Place,Bus Station,Butcher,Café,Caribbean Restaurant,Chinese Restaurant,Chocolate Shop,Clothing Store,Coffee Shop,Comfort Food Restaurant,Construction & Landscaping,Convenience Store,Cosmetics Shop,Cupcake Shop,Deli / Bodega,Department Store,Dim Sum Restaurant,Diner,Discount Store,Distribution Center,Dog Run,Electronics Store,Event Space,Fast Food Restaurant,Food & Drink Shop,Food Court,Food Service,Food Truck,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Furniture / Home Store,Gas Station,Gift Shop,Golf Course,Greek Restaurant,Grocery Store,Gym,Hockey Arena,Home Service,Hotel,Ice Cream Shop,Indian Restaurant,Intersection,Italian Restaurant,Japanese Restaurant,Jewelry Store,Juice Bar,Liquor Store,Lounge,Luggage Store,Massage Studio,Mediterranean Restaurant,Metro Station,Middle Eastern Restaurant,Miscellaneous Shop,Mobile Phone Shop,Movie Theater,Park,Pet Store,Pharmacy,Pizza Place,Plaza,Pool,Portuguese Restaurant,Pub,Ramen Restaurant,Restaurant,Salon / Barbershop,Sandwich Place,Shoe Store,Shopping Mall,Smoke Shop,Sporting Goods Shop,Steakhouse,Supermarket,Supplement Shop,Sushi Restaurant,Thai Restaurant,Theater,Toy / Game Store,Video Game Store,Vietnamese Restaurant,Women's Store
0,"Bathurst Manor, Wilson Heights, Downsview North",0.0,0.000000,0.000000,0.00,0.0,0.0,0.000000,0.0,0.090909,0.0,0.000000,0.0,0.00,0.00,0.0,0.000000,0.045455,0.0,0.0,0.0,0.0,0.000000,0.000000,0.00,0.00,0.0,0.00,0.090909,0.000000,0.0,0.0,0.0,0.000000,0.045455,0.0,0.00,0.045455,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.045455,0.045455,0.0,0.045455,0.045455,0.0,0.000000,0.045455,0.00,0.0,0.0,0.0,0.045455,0.000000,0.0,0.000000,0.000000,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.045455,0.0,0.0,0.0,0.045455,0.0,0.045455,0.045455,0.0,0.0,0.0,0.000000,0.0,0.045455,0.0,0.045455,0.0,0.045455,0.00,0.00,0.0,0.045455,0.0,0.045455,0.000000,0.0,0.0,0.0,0.0,0.0
1,Bayview Village,0.0,0.000000,0.000000,0.00,0.0,0.0,0.000000,0.0,0.250000,0.0,0.000000,0.0,0.00,0.00,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.000000,0.250000,0.00,0.25,0.0,0.00,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.000000,0.0,0.00,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,0.000000,0.0,0.000000,0.000000,0.0,0.000000,0.000000,0.00,0.0,0.0,0.0,0.000000,0.000000,0.0,0.000000,0.250000,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.000000,0.0,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0,0.000000,0.0,0.000000,0.00,0.00,0.0,0.000000,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0
2,"Bedford Park, Lawrence Manor East",0.0,0.000000,0.038462,0.00,0.0,0.0,0.000000,0.0,0.000000,0.0,0.000000,0.0,0.00,0.00,0.0,0.038462,0.000000,0.0,0.0,0.0,0.0,0.038462,0.038462,0.00,0.00,0.0,0.00,0.076923,0.038462,0.0,0.0,0.0,0.038462,0.000000,0.0,0.00,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,0.000000,0.0,0.000000,0.000000,0.0,0.038462,0.038462,0.00,0.0,0.0,0.0,0.000000,0.038462,0.0,0.076923,0.038462,0.0,0.038462,0.038462,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.000000,0.0,0.038462,0.076923,0.0,0.0,0.0,0.038462,0.0,0.038462,0.0,0.076923,0.0,0.000000,0.00,0.00,0.0,0.000000,0.0,0.038462,0.076923,0.0,0.0,0.0,0.0,0.0
3,Don Mills,0.0,0.000000,0.000000,0.04,0.0,0.0,0.040000,0.0,0.000000,0.0,0.000000,0.0,0.08,0.04,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.000000,0.040000,0.04,0.04,0.0,0.04,0.080000,0.000000,0.0,0.0,0.0,0.000000,0.000000,0.0,0.04,0.000000,0.040000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,0.000000,0.0,0.000000,0.000000,0.0,0.000000,0.000000,0.12,0.0,0.0,0.0,0.000000,0.000000,0.0,0.040000,0.080000,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.000000,0.0,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.0,0.080000,0.0,0.040000,0.0,0.00000

In [90]:
# Print each neighborhood along with the top 5 most common venues
num_top_venues = 5

for hood in NorthYork_grouped['Neighborhood']:
    print("----"+hood+"----")
    temp = NorthYork_grouped[NorthYork_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')


----Bathurst Manor, Wilson Heights, Downsview North----
         venue  freq
0         Bank  0.09
1  Coffee Shop  0.09
2  Gas Station  0.05
3  Bridal Shop  0.05
4    Gift Shop  0.05


----Bayview Village----
                 venue  freq
0   Chinese Restaurant  0.25
1                 Café  0.25
2                 Bank  0.25
3  Japanese Restaurant  0.25
4    Accessories Store  0.00


----Bedford Park, Lawrence Manor East----
                venue  freq
0  Italian Restaurant  0.08
1         Coffee Shop  0.08
2     Thai Restaurant  0.08
3      Sandwich Place  0.08
4         Pizza Place  0.08


----Don Mills----
                 venue  freq
0                  Gym  0.12
1          Coffee Shop  0.08
2           Restaurant  0.08
3  Japanese Restaurant  0.08
4           Beer Store  0.08


----Downsview----
                venue  freq
0       Grocery Store  0.23
1                Park  0.15
2       Shopping Mall  0.08
3             Airport  0.08
4  Athletics & Sports  0.08


----Fairview, Henry Fa

In [91]:
#put data into a pandas dataframe

def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [92]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = NorthYork_grouped['Neighborhood']

for ind in np.arange(NorthYork_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(NorthYork_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,"Bathurst Manor, Wilson Heights, Downsview North",Coffee Shop,Bank,Gas Station,Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Gift Shop,Diner,Grocery Store,Deli / Bodega
1,Bayview Village,Chinese Restaurant,Café,Bank,Japanese Restaurant,Women's Store,Discount Store,Construction & Landscaping,Convenience Store,Cosmetics Shop,Cupcake Shop
2,"Bedford Park, Lawrence Manor East",Coffee Shop,Thai Restaurant,Italian Restaurant,Pizza Place,Sandwich Place,Butcher,Restaurant,Juice Bar,Café,Japanese Restaurant
3,Don Mills,Gym,Coffee Shop,Japanese Restaurant,Beer Store,Restaurant,Sandwich Place,Clothing Store,Chinese Restaurant,Caribbean Restaurant,Café
4,Downsview,Grocery Store,Park,Bank,Athletics & Sports,Liquor Store,Shopping Mall,Baseball Field,Discount Store,Food Truck,Airport


**Cluster neighbourhoods**

In [93]:
# set number of clusters
kclusters = 5

NorthYork_grouped_clustering = NorthYork_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(NorthYork_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10]

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 3], dtype=int32)

In [94]:
# add clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

NorthYork_merged = NorthYork_data

# merge NorthYork_grouped with NorthYork_data to add latitude/longitude for each neighborhood
NorthYork_merged = NorthYork_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')

NorthYork_merged

,Postal Code,Latitude,Longitude,Borough,Neighborhood,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M2H,43.803762,-79.363452,North York,Hillcrest Village,0.0,Golf Course,Mediterranean Restaurant,Fast Food Restaurant,Pool,Dog Run,Women's Store,Dim Sum Restaurant,Coffee Shop,Comfort Food Restaurant,Construction & Landscaping
1,M2J,43.778517,-79.346556,North York,"Fairview, Henry Farm, Oriole",0.0,Clothing Store,Coffee Shop,Fast Food Restaurant,Shoe Store,Restaurant,Juice Bar,Japanese Restaurant,Bank,Bakery,Women's Store
2,M2K,43.786947,-79.385975,North York,Bayview Village,0.0,Chinese Restaurant,Café,Bank,Japanese Restaurant,Women's Store,Discount Store,Construction & Landscaping,Convenience Store,Cosmetics Shop,Cupcake Shop
3,M2L,43.757490,-79.374714,North York,"York Mills, Silver Hills",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,M2M,43.789053,-79.408493,North York,"Willowdale, Newtonbrook",1.0,Park,Chocolate Shop,Clothing Store,Coffee Shop,Comfort Food Restaurant,Construction & Landscaping,Convenience Store,Cosmetics Shop,Cupcake Shop,Deli / Bodega
5,M2N,43.770120,-79.408493,North York,"Willowdale, Willowdale East",0.0,Ramen Restaurant,Pizza Place,Café,Restaurant,Sushi Restaurant,Sandwich Place,Coffee Shop,Shopping Mall,Movie Theater,Pet Store
6,M2P,43.752758,-79.400049,North York,York Mills West,4.0,Park,Convenience Store,Chocolate Shop,Clothing Store,Coffee Shop,Comfort Food Restaurant,Construction & Landscaping,Cosmetics Shop,Cupcake Shop,Deli / Bodega
7,M2R,43.782736,-79.442259,North York,"Willowdale, Willowdale West",0.0,Discount Store,Coffee Shop,Pharmacy,Pizza Place,Grocery Store,Diner,Comfort Food Restaurant,Construction & Landscaping,Convenience Store,Cosmetics Shop
8,M3A,43.753259,-79.329656,North York,Parkwoods,2.0,Park,Food & Drink Shop,Diner,Clothing Store,Coffee Shop,Comfort Food Restaurant,Construction & Landscaping,Convenience Store,Cosmetics Shop,Cupcake Shop
9,M3B,43.745906,-79.352188,North York,Don Mills,0.0,Gym,Coffee Shop,Japanese Restaurant,Beer Store,Restaurant,Sandwich Place,Clothing Store,Chinese Restaurant,Caribbean Restaurant,Café


In [97]:
#add a column to the dataframe that indicates how many times a restaurant/cafe/eatery appears as top 10 common venue in each neighbourhood
#I calculated the count manually (by looking at the table and keeping count)

NorthYork_merged['Restaurant Count'] = ['5','6','4','n/a','6','7','5','4','5','6','6','6','4','1','1','1','1','8','9','3','5','3','4','3']

In [98]:
NorthYork_merged

,Postal Code,Latitude,Longitude,Borough,Neighborhood,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue,Restaurant Count
0,M2H,43.803762,-79.363452,North York,Hillcrest Village,0.0,Golf Course,Mediterranean Restaurant,Fast Food Restaurant,Pool,Dog Run,Women's Store,Dim Sum Restaurant,Coffee Shop,Comfort Food Restaurant,Construction & Landscaping,5
1,M2J,43.778517,-79.346556,North York,"Fairview, Henry Farm, Oriole",0.0,Clothing Store,Coffee Shop,Fast Food Restaurant,Shoe Store,Restaurant,Juice Bar,Japanese Restaurant,Bank,Bakery,Women's Store,6
2,M2K,43.786947,-79.385975,North York,Bayview Village,0.0,Chinese Restaurant,Café,Bank,Japanese Restaurant,Women's Store,Discount Store,Construction & Landscaping,Convenience Store,Cosmetics Shop,Cupcake Shop,4
3,M2L,43.757490,-79.374714,North York,"York Mills, Silver Hills",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,n/a
4,M2M,43.789053,-79.408493,North York,"Willowdale, Newtonbrook",1.0,Park,Chocolate Shop,Clothing Store,Coffee Shop,Comfort Food Restaurant,Construction & Landscaping,Convenience Store,Cosmetics Shop,Cupcake Shop,Deli / Bodega,6
5,M2N,43.770120,-79.408493,North York,"Willowdale, Willowdale East",0.0,Ramen Restaurant,Pizza Place,Café,Restaurant,Sushi Restaurant,Sandwich Place,Coffee Shop,Shopping Mall,Movie Theater,Pet Store,7
6,M2P,43.752758,-79.400049,North York,York Mills West,4.0,Park,Convenience Store,Chocolate Shop,Clothing Store,Coffee Shop,Comfort Food Restaurant,Construction & Landscaping,Cosmetics Shop,Cupcake Shop,Deli / Bodega,5
7,M2R,43.782736,-79.442259,North York,"Willowdale, Willowdale West",0.0,Discount Store,Coffee Shop,Pharmacy,Pizza Place,Grocery Store,Diner,Comfort Food Restaurant,Construction & Landscaping,Convenience Store,Cosmetics Shop,4
8,M3A,43.753259,-79.329656,North York,Parkwoods,2.0,Park,Food & Drink Shop,Diner,Clothing Store,Coffee Shop,Comfort Food Restaurant,Construction & Landscaping,Convenience Store,Cosmetics Shop,Cupcake Shop,5
9,M3B,43.745906,-79.352188,North York,Don Mills,0.0,Gym,Coffee Shop,Japanese Restaurant,Beer Store,Restaurant,Sandwich Place,Clothing Store,Chinese Restaurant,Caribbean Restaurant,Café,6


### Business decision
By examining the table above (looking at the "Restaurant Count' column), I can conclude that Downsview has the least number of restaurants/cafes/eateries as a common venue. Therefore, my client should open a Japanese restaurant in **Downsview**.